In [ ]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.8 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=884847501ecd9ece554ef5aca1383ee8c7bc958408434271b0780519bc61877f
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 68.2 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=4d1a1664cc6ffdbaa8e19939cd43e4239047fc0db4885e0c41c5365c0f670178
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [ ]:
%%writefile app.py
import mlflow
import streamlit as st
import warnings 
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

# Define Streamlit UI components
st.title("Categorical Prediction")
train_data = st.file_uploader("Upload a train file")
test_data = st.file_uploader("Upload a test file")

# Log file on upload
if train_data is not None:
    train_df = pd.read_csv(train_data)
    if test_data is not None:
        test_df = pd.read_csv(test_data)

        # Encode categorical features
        encoder = LabelEncoder()
        for col in train_df.select_dtypes(include=['object']):
            train_df[col] = encoder.fit_transform(train_df[col])
            test_df[col] = encoder.transform(test_df[col])

        # Prepare data for modeling
        X_train = train_df.drop(['id', 'target'], axis=1)
        y_train = train_df['target']
        X_test = test_df.drop('id', axis=1)

        scale = StandardScaler().fit(X_train)
        X_train_scaled = scale.transform(X_train)

        # Fit Decision Tree Regressor model
        with mlflow.start_run():
            model = DecisionTreeRegressor(random_state=0)
            model.fit(X_train_scaled, y_train)

            # Log model parameters
            mlflow.log_param("max_depth", model.get_params()['max_depth'])
            mlflow.log_param("min_samples_split", model.get_params()['min_samples_split'])
            mlflow.log_param("min_samples_leaf", model.get_params()['min_samples_leaf'])
            mlflow.log_param("criterion", model.get_params()['criterion'])
            mlflow.log_param("splitter", model.get_params()['splitter'])
            mlflow.log_param("model", "DecisionTreeRegressor")

            X_test_scaled = scale.transform(X_test)
            X_test_arr = np.array(X_test_scaled)

            # Predict target for test set
            y_test_pred = model.predict(X_test_arr)

            # Calculate RMSE
            y_train_pred = model.predict(X_train_scaled)
            train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
            mlflow.log_metric("train_rmse", train_rmse)

            submission = test_df[['id']].copy()
            submission['target'] = y_test_pred
            csv_file = submission.to_csv(index=False)

            st.download_button('Download file', data=csv_file, file_name='submission.csv', mime='text/csv')


Writing app.py


In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2NmpozMseEJY59TXYP8fnAQABm3_6xQru6WAQwTcQJFcRa8G3")

In [ ]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port='80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://466c-35-196-252-221.ngrok.io" -> "http://localhost:80"
